# What is Named Entity Recognition ? 

Es un problema de *Sequence Labeling*.  A diferencia de obtener una clase para todo un texto, cada token del texto debe ser taggeado.

### Algo de teoría breve

#### Sequence Labeling & Structured Output

Al ser un proceso de etiquetar sequencias, que puede ser mejor, etiquetar toda la sequencia de golpe, o ir palabra a palabra?

Bueno, con lo que hemos visto hasta ahora, tanto en Machine Learning, como en Deep Learning, no hay manera de etiquetar todo de golpe, porque normalmente tenemos datasets tal que las *ys* dependen unicamente de un set de features representado por *x*. En NLP muchos de estos problemas no tienen sentido, tal y como hemos visto en Language Modeling, hay casos en los que la palabra actual depende, tanto de lo que hemos dicho(pasado), como de lo que vamos a decir (futuro). De hecho, cuando vamos a hablar o escribir, *normalmente y no todos* pensamos que vamos a decir, es decir, hacemos una predicción de aquello futuro, y luego vamos articulando esa idea (pasado). Para estos casos, hay modelos probabilísticos como los modelos markovianos de variables latentes (Hidden Markov Models [HMM](https://en.wikipedia.org/wiki/Hidden_Markov_model)), que quedan fuera del temario del curso, o Conditional Random Fields ([CRF](https://en.wikipedia.org/wiki/Conditional_random_field)) como su alternativa más usada para problemas de este estilo. 

Este último, y solo a modo de que os suene, cae en la categoría de *structured prediction* ([wiki](https://en.wikipedia.org/wiki/Structured_prediction)). Structured prediction no es ni más ni menos que lo que nosotros querríamos para este tipo de productos, es decir, en lugar de que *y* este condicionada sólo a una serie de *features x*, este condicionado a todo el output. Y que todo este entrelazado, que por ejemplo un tag posterior, pueda afectar a uno anterior y viceversa.

![](https://i.imgur.com/ukAr3Uh.jpg)

Entonces si no vemos esto dentro del curso, hemos terminado la clase porque no podemos resolver el problema. Bueno no del todo.

Como siempre, hay alternativas, que quizás no cumplen con todos los requisitos que querriamos pero dan resultados correctos, sobretodo para ver baselines.

Entonces, si no podemos predecir algo en base al tiempo, haremos que el tiempo sea una feature. Es decir, en una feature meteremos tambien las features de *t-n* hasta *t+n*. Es decir, que la feature *x_n* estará compuesta por los la concatenación de elementos de features pasadas, y features futuras, y haremos que el clasificador tenga suficiente información en cada pareja *x,y*. Veremos en breves como se componen estas features.

![](https://i.imgur.com/Wail7yI.jpg =400x)
![](https://i.imgur.com/irjqooW.jpg =400x)

De esta forma, conseguimos crear features "independientes" del tiempo y podremos entrenar como siempre.

### Named Entity Recognition

El Objectivo es asignar doble-tag uno para cada clase, persona, organizacion, lo que queramos, y el otro es el conocido como BIO.

*   Begin: Inicio de una entidad
*   Inside: Dentro de una entidad
*   Outside: No es una entida

Actualmente y bastantes taggers correctos, pero en general, hay que adaptarlo al dominio. Un caso muy claro, es el de esos taggers, como el que veremos que se basa entre otras features, en si una palabra esta en mayúsculas. En un dominio más casual, como twitter, o cualquier otro ambito del tipo de internet.

![](https://i.imgur.com/a8Zregf.jpg)

Cual es la idea para entrenar? Para todo un texto usamos n-grams fomo features normalmente, entre otras. Aqui la idea es palabra por palabra poder decidir que tag tiene. Se definen una seria de features, que no son n-grams. Ahoar veremos un subset de estas features. Para lo demas, sera como cualquier otro modelo. Representar como input las features, tener preparado para cada input, un output, escoger clasificador y función de objetivo y a entrenar!

In [0]:
import csv
from tqdm import tqdm
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])

In [1]:
#collab opening

dataset = []

with open('ner_dataset.csv', encoding='mac_roman',) as f:
    dataset_ner = csv.reader(f)
    next(dataset_ner, None)  
    sentence = []    
    for row in dataset_ner: # .encode('utf-8').strip()
        if row[0] != '':
            dataset.append(sentence)
            sentence = [(row[1],row[2], row[3])]
        else:
            sentence.append((row[1],row[2], row[3]))
dataset.remove([])
len(dataset)

FileNotFoundError: ignored

In [0]:
dataset[0]

[('Thousands', 'NNS', 'O'),
 ('of', 'IN', 'O'),
 ('demonstrators', 'NNS', 'O'),
 ('have', 'VBP', 'O'),
 ('marched', 'VBN', 'O'),
 ('through', 'IN', 'O'),
 ('London', 'NNP', 'B-geo'),
 ('to', 'TO', 'O'),
 ('protest', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('war', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('Iraq', 'NNP', 'B-geo'),
 ('and', 'CC', 'O'),
 ('demand', 'VB', 'O'),
 ('the', 'DT', 'O'),
 ('withdrawal', 'NN', 'O'),
 ('of', 'IN', 'O'),
 ('British', 'JJ', 'B-gpe'),
 ('troops', 'NNS', 'O'),
 ('from', 'IN', 'O'),
 ('that', 'DT', 'O'),
 ('country', 'NN', 'O'),
 ('.', '.', 'O')]

In [0]:
classes = list(set([c for sentence in dataset for (_, _, c) in sentence]))
classes

['B-org',
 'I-per',
 'B-geo',
 'I-org',
 'I-nat',
 'B-eve',
 'B-gpe',
 'B-nat',
 'I-tim',
 'I-eve',
 'I-art',
 'O',
 'I-geo',
 'B-per',
 'B-art',
 'I-gpe',
 'B-tim']

# Building sets of features

Hasta ahora, hemos usado como features solo una minúscula parte del potencial linguístico que tienen los textos. Para este ejercicio entrenaremos con nuevas features, algunas muy obvias, otras quizas no tanto. En la imagen se pueden ver features para crear un Part of Speech tagger, nosotros lo adaptaremos a un Named Entity Recognizer.

![alt text](https://i.imgur.com/1SAC0TU.jpg =500x)

From Neural Network Methods for NLP by Yoav Goldberg


Acordemonos que las features hay que adaptarlas no solo al problema, sino al dataset. El problema nos puede dar un estimado de que features usar, pero como siempre, nosotros podemos generar, quitar o combinar features como queramos. Por ejemplo, en el case de Entity Recognizers, que una palabra este en mayúsculas quizás es muy relevante, pero si estamos en un contexto de internet, donde la gente no sigue una convención fija de escriptura, es menos relevante.

Para que veais un ejemplo de la complejidad de la tarea.




> Mal
![](https://i.imgur.com/wCOsdsk.png)
![](https://i.imgur.com/EO89nLZ.png)

> Bien
![](https://i.imgur.com/amsQr9T.png)

In [0]:
import re
 
def shape(word):
    word_shape = 'other'
    if re.match('[0-9]+(\.[0-9]*)?|[0-9]*\.[0-9]+$', word):
        word_shape = 'number'
    elif re.match('\W+$', word):
        word_shape = 'punct'
    elif re.match('[A-Z][a-z]+$', word):
        word_shape = 'capitalized'
    elif re.match('[A-Z]+$', word):
        word_shape = 'uppercase'
    elif re.match('[a-z]+$', word):
        word_shape = 'lowercase'
    elif re.match('[A-Z][a-z]+[A-Z][a-z]+[A-Za-z]*$', word):
        word_shape = 'camelcase'
    elif re.match('[A-Za-z]+$', word):
        word_shape = 'mixedcase'
    elif re.match('__.+__$', word):
        word_shape = 'wildcard'
    elif re.match('[A-Za-z0-9]+\.$', word):
        word_shape = 'ending-dot'
    elif re.match('[A-Za-z0-9]+\.[A-Za-z0-9\.]+\.$', word):
        word_shape = 'abbreviation'
    elif re.match('[A-Za-z0-9]+\-[A-Za-z0-9\-]+.*$', word):
        word_shape = 'contains-hyphen'
 
    return word_shape

In [0]:
def ner_features(tokens, index, history):
    """
    `tokens`  = a POS-tagged sentence [(w1, t1), ...]
    `index`   = the index of the token we want to extract features for
    `history` = the previous predicted IOB tags
    """
    # Pad the sequence with placeholders
    tokens = [('__START2__', '__START2__','__START2__' ), ('__START1__', '__START1__', '__START1__')] + list(tokens) + [('__END1__', '__END1__', '__END1__'), ('__END2__', '__END2__', '__END2__')]
    history = ['__START2__', '__START1__'] + list(history)
    # shift the index with 2, to accommodate the padding
    index += 2
    word, pos, lemma = tokens[index]
    prevword, prevpos, prevlemma = tokens[index - 1]
    prevprevword, prevprevpos, prevprevlemma = tokens[index - 2]
    nextword, nextpos, nextlemma = tokens[index + 1]
    nextnextword, nextnextpos, nextnextlemma = tokens[index + 2]
    previob = history[-1]
    prevpreviob = history[-2]
 
    feat_dict = {
        'word': word,
        'lemma': lemma,
        'pos': pos,
        'shape': shape(word),
 
        'next-word': nextword,
        'next-pos': nextpos,
        'next-lemma': nextlemma,
        'next-shape': shape(nextword),
 
        'next-next-word': nextnextword,
        'next-next-pos': nextnextpos,
        'next-next-lemma': nextnextlemma,
        'next-next-shape': shape(nextnextword),
 
        'prev-word': prevword,
        'prev-pos': prevpos,
        'prev-lemma': prevlemma,
        'prev-iob': previob,
        'prev-shape': shape(prevword),
 
        'prev-prev-word': prevprevword,
        'prev-prev-pos': prevprevpos,
        'prev-prev-lemma': prevprevlemma,
        'prev-prev-iob': prevpreviob,
        'prev-prev-shape': shape(prevprevword),
    }
 
    return feat_dict
    

In [0]:
def to_dataset(parsed_sentences, feature_detector):
        """
        Transform a list of tagged sentences into a scikit-learn compatible POS dataset
        :param parsed_sentences:
        :param feature_detector:
        :return:
        """
        X, y = [], []
        for parsed in tqdm(parsed_sentences):
            words, tags, iob_tags = zip(*parsed)
            lemmas = [t.lemma_ for t in nlp(" ".join(words))]
            for index in range(len(parsed)):
                tagged = zip(words, tags, lemmas)
                X.append(feature_detector(tagged, index, history=iob_tags[:index]))
                y.append(iob_tags[index])
        return X, y

In [0]:
X, y = to_dataset(dataset[0:5000], ner_features)

100%|██████████| 5000/5000 [01:06<00:00, 75.08it/s]


In [0]:
import itertools
def get_minibatch(parsed_sentences, feature_detector, batch_size=500):
    batch = list(itertools.islice(parsed_sentences, batch_size))
    X, y = to_dataset(batch, feature_detector)
    return X, y

In [0]:
X[0]

{'lemma': 'thousand',
 'next-lemma': 'of',
 'next-next-lemma': 'demonstrator',
 'next-next-pos': 'NNS',
 'next-next-shape': 'lowercase',
 'next-next-word': 'demonstrators',
 'next-pos': 'IN',
 'next-shape': 'lowercase',
 'next-word': 'of',
 'pos': 'NNS',
 'prev-iob': '__START1__',
 'prev-lemma': '__START1__',
 'prev-pos': '__START1__',
 'prev-prev-iob': '__START2__',
 'prev-prev-lemma': '__START2__',
 'prev-prev-pos': '__START2__',
 'prev-prev-shape': 'wildcard',
 'prev-prev-word': '__START2__',
 'prev-shape': 'wildcard',
 'prev-word': '__START1__',
 'shape': 'capitalized',
 'word': 'Thousands'}

In [0]:
from sklearn.linear_model import Perceptron
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

In [0]:
vectorizer = DictVectorizer(sparse=False)


In [0]:
ner_clf = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', Perceptron(verbose=10, n_jobs=-1, n_iter=5))
])


In [0]:
X, y = get_minibatch(dataset, ner_features, 500)
vectorizer.fit(X)
while len(X):
    X = vectorizer.transform(X)
    clf.partial_fit(X, y, classes)
    X, y = get_minibatch(dataset, ner_features, 500)

100%|██████████| 500/500 [00:06<00:00, 75.05it/s]


-- Epoch 1-- Epoch 1

-- Epoch 1-- Epoch 1

Norm: 16.91, NNZs: 231, Bias: -4.000000, T: 10976, Avg. loss: 0.009293
Total training time: 0.53 seconds.
-- Epoch 1
Norm: 54.09, NNZs: 1445, Bias: -9.000000, T: 10976, Avg. loss: 0.134384
Total training time: 0.59 seconds.
-- Epoch 1
Norm: 62.13, NNZs: 1837, Bias: -9.000000, T: 10976, Avg. loss: 0.165179
Total training time: 0.61 seconds.
-- Epoch 1
Norm: 25.38, NNZs: 480, Bias: -3.000000, T: 10976, Avg. loss: 0.025784
Total training time: 0.62 seconds.
-- Epoch 1
Norm: 14.70, NNZs: 150, Bias: -3.000000, T: 10976, Avg. loss: 0.003371
Total training time: 0.56 seconds.
-- Epoch 1
Norm: 52.57, NNZs: 1439, Bias: -10.000000, T: 10976, Avg. loss: 0.114705
Total training time: 0.57 seconds.
-- Epoch 1
Norm: 45.89, NNZs: 1130, Bias: -6.000000, T: 10976, Avg. loss: 0.070062
Total training time: 0.55 seconds.
-- Epoch 1
Norm: 48.79, NNZs: 1070, Bias: -6.000000, T: 10976, Avg. loss: 0.065962
Total training time: 0.59 seconds.
-- Epoch 1


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.1s


Norm: 25.14, NNZs: 387, Bias: -4.000000, T: 10976, Avg. loss: 0.010295
Total training time: 0.56 seconds.
-- Epoch 1
Norm: 18.44, NNZs: 228, Bias: -4.000000, T: 10976, Avg. loss: 0.004738
Total training time: 0.54 seconds.
-- Epoch 1
Norm: 25.57, NNZs: 375, Bias: -5.000000, T: 10976, Avg. loss: 0.014122
Total training time: 0.55 seconds.
-- Epoch 1
Norm: 24.25, NNZs: 376, Bias: -5.000000, T: 10976, Avg. loss: 0.010842
Total training time: 0.59 seconds.
-- Epoch 1


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  12 out of  17 | elapsed:    1.8s remaining:    0.8s


...

Norm: 50.06, NNZs: 1049, Bias: -9.000000, T: 10976, Avg. loss: 0.000000
Total training time: 0.54 seconds.
-- Epoch 1
Norm: 10.86, NNZs: 97, Bias: -3.000000, T: 10976, Avg. loss: 0.000000
Total training time: 0.79 seconds.
Norm: 36.36, NNZs: 624, Bias: -7.000000, T: 10976, Avg. loss: 0.000000
Total training time: 0.73 seconds.
Norm: 19.60, NNZs: 253, Bias: -4.000000, T: 10976, Avg. loss: 0.000000
Total training time: 0.78 seconds.


[Parallel(n_jobs=-1)]: Done  14 out of  17 | elapsed:    2.5s remaining:    0.5s
[Parallel(n_jobs=-1)]: Done  17 out of  17 | elapsed:    2.7s finished
  1%|          | 5/500 [00:00<00:10, 48.58it/s]

Norm: 91.52, NNZs: 2278, Bias: 3.000000, T: 10976, Avg. loss: 0.000911
Total training time: 0.50 seconds.


100%|██████████| 500/500 [00:06<00:00, 72.00it/s]


KeyboardInterrupt: 

add results visualization